In [2]:
import sys
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding
import os
import numpy as np
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding, Flatten,Merge,dot,multiply,concatenate,Lambda,Dropout
from keras.layers.advanced_activations import PReLU
from keras.models import Model
from keras import backend as K
K.set_image_dim_ordering('tf')
from numpy import array
from numpy import argmax
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras.models import model_from_json
MAX_SEQUENCE_LENGTH = 78
GLOVE_DIR = ""
EMBEDDING_DIM = 200

Using TensorFlow backend.


In [3]:
with open("/home/shyam/Downloads/MS/Sem1/NLP/Project/Data/Seprated/train_s1_.txt", "r") as ins:
    texts_s1 = []
    for line in ins:
        line = line.strip('\n')
        line = line.strip('\t')
        texts_s1.append(line)
        
with open("/home/shyam/Downloads/MS/Sem1/NLP/Project/Data/Seprated/train_s2_.txt", "r") as ins:
    texts_s2 = []
    for line in ins:
        line = line.strip('\n')
        line = line.strip('\t')
        texts_s2.append(line)
with open("/home/shyam/Downloads/MS/Sem1/NLP/Project/Data/Seprated/train_label_.txt", "r") as ins:
    label = []
    for line in ins:
        line = line.strip('\n')
        line = line.strip('\t')
        label.append(int(line))       
        
PN_label = []
PN_text_s1 = []
PN_text_s2 = []
Count = 0;
for i in range(0,len(label)):
    if (label[i])==2 or (label[i])==3:
        Count = Count + 1
        PN_label.append(label[i])
        PN_text_s1.append(texts_s1[i])
        PN_text_s2.append(texts_s2[i])
    if Count == 100000:
        break;
print len(PN_label)

100000


In [4]:
with open("/home/shyam/Downloads/MS/Sem1/NLP/Project/Data/Seprated2/train_s_.txt", "r") as ins:
    texts = []
    for line in ins:
        line = line.strip('\n')
        line = line.strip('\t')
        texts.append(line)
        
PN_text = []
for i in range(0,len(label)/2):
    if (label[i])==2 or (label[i])==3:
        PN_text.append(texts[i])
        
tokenizer = Tokenizer()
tokenizer.fit_on_texts(PN_text)

GLOVE_DIR = "/home/shyam/Downloads/MS/Sem1/NLP/Project/glove/glove.6B/"
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.200d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

word_index = tokenizer.word_index
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
count1 = 0
count2 = 0
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        count1 = count1 +1
    else:
        count2 = count2 +1

In [5]:
sequences1 = tokenizer.texts_to_sequences(PN_text_s1)
sequences2 = tokenizer.texts_to_sequences(PN_text_s2)

data1 = pad_sequences(sequences1, maxlen=MAX_SEQUENCE_LENGTH)
data2 = pad_sequences(sequences2, maxlen=MAX_SEQUENCE_LENGTH)

label = np.asarray(PN_label)  
label[:] = [x - 2 for x in label] 
label = to_categorical(label)

In [6]:
with open("/home/shyam/Downloads/MS/Sem1/NLP/Project/Data/Seprated/test_s1_.txt", "r") as ins:
    dev_texts_s1 = []
    for line in ins:
        line = line.strip('\n')
        line = line.strip('\t')
        dev_texts_s1.append(line)
        
with open("/home/shyam/Downloads/MS/Sem1/NLP/Project/Data/Seprated/test_s2_.txt", "r") as ins:
    dev_texts_s2 = []
    for line in ins:
        line = line.strip('\n')
        line = line.strip('\t')
        dev_texts_s2.append(line)
with open("/home/shyam/Downloads/MS/Sem1/NLP/Project/Data/Seprated/test_label_.txt", "r") as ins:
    dev_label = []
    for line in ins:
        line = line.strip('\n')
        line = line.strip('\t')
        dev_label.append(int(line))

dev_PN_label = []
dev_PN_text_s1 = []
dev_PN_text_s2 = []

for i in range(0,len(dev_label)):
    if (dev_label[i])==2 or (dev_label[i])==3:
        dev_PN_label.append(dev_label[i])
        dev_PN_text_s1.append(dev_texts_s1[i])
        dev_PN_text_s2.append(dev_texts_s2[i])
dev_sequences1 = tokenizer.texts_to_sequences(dev_PN_text_s1)
dev_sequences2 = tokenizer.texts_to_sequences(dev_PN_text_s2)

dev_data1 = pad_sequences(dev_sequences1, maxlen=MAX_SEQUENCE_LENGTH)
dev_data2 = pad_sequences(dev_sequences2, maxlen=MAX_SEQUENCE_LENGTH)

dev_label = np.asarray(dev_PN_label)  
dev_label[:] = [x - 2 for x in dev_label] 
#dev_label = to_categorical(dev_label)

In [7]:
def _differences(pair_of_tensors):
    x, y = pair_of_tensors
    return (x - y)

In [8]:
Input1 = Input(shape=(MAX_SEQUENCE_LENGTH,),dtype='int32')
Input2 = Input(shape=(MAX_SEQUENCE_LENGTH,),dtype='int32')

sentence1 = Embedding(len(word_index) + 1,EMBEDDING_DIM, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH, 
                 trainable=False)(Input1)

sentence2 = Embedding(len(word_index) + 1,EMBEDDING_DIM,weights=[embedding_matrix],input_length=MAX_SEQUENCE_LENGTH, 
                 trainable=False)(Input2)

convs_s1 = []
filter_sizes = [2,3,4,5]

for fsz in filter_sizes:
    l_conv_s1 = Conv1D(nb_filter=128,filter_length=fsz,activation='relu')(sentence1)
    #l_conv_s1 = Dropout(0.2)(l_conv_s1)
    l_pool_s1 = MaxPooling1D(5)(l_conv_s1)
    convs_s1.append(l_pool_s1)

l_merge_s1 = Merge(mode='concat', concat_axis=1)(convs_s1)
#l_merge_s1 = BatchNormalization()(l_merge_s1)
l_cov1_s1= Conv1D(128, 5, activation='relu')(l_merge_s1)
l_pool1_s1 = MaxPooling1D(5)(l_cov1_s1)
#l_pool1_s1 = BatchNormalization()(l_pool1_s1)
l_cov2_s1 = Conv1D(192, 5, activation='relu')(l_pool1_s1)
l_pool2_s1 = MaxPooling1D(5)(l_cov2_s1)
#l_pool2_s1 = BatchNormalization()(l_pool2_s1)
convs_s2 = []

for fsz in filter_sizes:
    l_conv_s2 = Conv1D(nb_filter=128,filter_length=fsz,activation='relu')(sentence2)
 #   l_conv_s2 = Dropout(0.2)(l_conv_s2)
    l_pool_s2 = MaxPooling1D(5)(l_conv_s2)
    convs_s2.append(l_pool_s2)

l_merge_s2 = Merge(mode='concat', concat_axis=1)(convs_s2)
#l_merge_s2 = BatchNormalization()(l_merge_s2)
l_cov1_s2= Conv1D(128, 5, activation='relu')(l_merge_s2)
l_pool1_s2 = MaxPooling1D(5)(l_cov1_s2)
#l_pool1_s2 = BatchNormalization()(l_pool1_s2)
l_cov2_s2 = Conv1D(192, 5, activation='relu')(l_pool1_s2)
l_pool2_s2 = MaxPooling1D(5)(l_cov2_s2)
#l_pool2_s2 = BatchNormalization()(l_pool2_s2)

merge_m = multiply([l_cov2_s2,l_cov2_s1])
merge_d = Lambda(_differences)([l_cov2_s2, l_cov2_s1])
merge_s = concatenate([merge_d,merge_m],axis = -2)
#merge_s = BatchNormalization()(merge_s)
merge_s = Flatten()(merge_s)
merge_s = Dense(2048, activation='relu')(merge_s)
merge_s = Dropout(0.1)(merge_s)
merge_s = Dense(512, activation='relu')(merge_s)
merge_s = Dropout(0.05)(merge_s)
merge_s = Dense(84, activation='relu')(merge_s)
preds = Dense(2, activation='softmax')(merge_s)


model = Model(inputs=[Input1,Input2], outputs=preds)
lr = 0.0003
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=lr, decay = lr*0.95),
              metrics=['acc'])

/home/shyam/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:14: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", kernel_size=2, filters=128)`
/home/shyam/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:14: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", kernel_size=3, filters=128)`
/home/shyam/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:14: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", kernel_size=4, filters=128)`
/home/shyam/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:14: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", kernel_size=5, filters=128)`
/home/shyam/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:19: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/h

In [12]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

In [62]:
#model.fit([data1,data2],label,validation_split=0.01,nb_epoch=5, batch_size=64)
import datetime, time, json
from keras.callbacks import Callback, ModelCheckpoint
from keras.models import Model
#print("Starting training at", datetime.datetime.now())
#t0 = time.time()
#callbacks = [ModelCheckpoint(MODEL_WEIGHTS_FILE, monitor='val_acc', save_best_only=True)]
model.fit([data1,data2],
                    label,
                    epochs=7,
                    validation_split=0.01,
                    shuffle=True,
                    batch_size=128)
#t1 = time.time()
#print("Training ended at", datetime.datetime.now())
#print("Minutes elapsed: %f" % ((t1 - t0) / 60.))

Train on 99000 samples, validate on 1000 samples
Epoch 1/7
99000/99000 [==============================] - 1960s - loss: 0.5485 - acc: 0.7081 - val_loss: 0.4863 - val_acc: 0.7480
Epoch 2/7
99000/99000 [==============================] - 2102s - loss: 0.4684 - acc: 0.7671 - val_loss: 0.4585 - val_acc: 0.7720
Epoch 3/7
99000/99000 [==============================] - 1979s - loss: 0.4251 - acc: 0.7962 - val_loss: 0.4522 - val_acc: 0.7710
Epoch 4/7
99000/99000 [==============================] - 1955s - loss: 0.3773 - acc: 0.8248 - val_loss: 0.4316 - val_acc: 0.8000
Epoch 5/7
99000/99000 [==============================] - 1957s - loss: 0.3322 - acc: 0.8508 - val_loss: 0.4273 - val_acc: 0.8030
Epoch 6/7
99000/99000 [==============================] - 1916s - loss: 0.2899 - acc: 0.8742 - val_loss: 0.4441 - val_acc: 0.8100
Epoch 7/7
99000/99000 [==============================] - 1894s - loss: 0.2469 - acc: 0.8962 - val_loss: 0.4747 - val_acc: 0.8120


In [72]:
model_json = model.to_json()
with open("model_V2.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("model_V2.h5")
print("Saved model to disk")
  
json_file = open("model_V2.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights("model_V2.h5")
print("Loaded model from disk")
 
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
Prediction =  np.argmax(loaded_model.predict([dev_data1,dev_data2]), axis = 1)
print float(sum(dev_label==Prediction))/len(dev_label)*100

Saved model to disk
Loaded model from disk
5289


80.0757002271
